### Model Training

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/RTA-LABS/datasets/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop(labels=['id'], axis=1, inplace=True)

In [4]:
## Independent and dependent features

X = df.drop(labels=['price'], axis=1)
y = df[['price']]

In [5]:
# Define which column should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [6]:
# Define custom ranking for each ordinal variable

cut_categoris = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [7]:
from sklearn.impute import SimpleImputer # Handling the missing values
from sklearn.preprocessing import StandardScaler # Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # Merge the pipelines

In [11]:
# Numerical Pipeline

num_pipeline = Pipeline(
    steps=[
        ('impute', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical Pipeline

cat_pipeline = Pipeline(
    steps = [
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categoris, color_categories, clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

In [12]:
# Train Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((135501, 9), (58072, 9), (135501, 1), (58072, 1))

In [14]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [37]:
# Model Training

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [17]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [18]:
regression.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [19]:
regression.intercept_

array([3976.8787389])

In [40]:
import numpy as np

def evaluate_model(y_true, y_pred):
  mse = mean_squared_error(y_true, y_pred)
  rmse = rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  mae = mean_absolute_error(y_true, y_pred)
  r2_square = r2_score(y_true, y_pred)
  return mse, rmse, mae, r2_square

In [41]:
# Training Multiple Models

models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "ElasticNet" : ElasticNet()
}

trained_models = {}
model_list = []
r2_list = []

for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(X_train, y_train)

  # Make Predictions

  y_pred = model.predict(X_test)

  mse, mae,rmse, r2_square = evaluate_model(y_test, y_pred)
  print(list(models.keys())[i])
  model_list.append(list(models.keys())[i])
  trained_models.update({list(models.keys())[i] : model})

  print("Model Training Performance")
  print("MSE:",mse)
  print("RMSE:",rmse)
  print("MAE:",mae)
  print("R2 score",r2_square*100)

  r2_list.append(r2_square)

  print("="*40)
  print()

LinearRegression
Model Training Performance
MSE: 1029473.3531156889
RMSE: 675.0758270067445
MAE: 1014.6296630375483
R2 score 93.62906819996046

Lasso
Model Training Performance
MSE: 1029533.1506505491
RMSE: 676.2421173665509
MAE: 1014.659130275064
R2 score 93.62869814082755

Ridge
Model Training Performance
MSE: 1029482.8101269199
RMSE: 675.1077629781057
MAE: 1014.6343233534533
R2 score 93.62900967491616

ElasticNet
Model Training Performance
MSE: 2351174.8713978743
RMSE: 1060.9432977143008
MAE: 1533.3541245902313
R2 score 85.44967219374031



In [42]:
trained_models

{'LinearRegression': LinearRegression(),
 'Lasso': Lasso(),
 'Ridge': Ridge(),
 'ElasticNet': ElasticNet()}

In [43]:
lasso = trained_models.get("Lasso")

In [44]:
lasso.coef_

array([ 6388.83205065,  -129.94146246,   -66.55952582, -1999.8773794 ,
        -172.61296695,   -56.60669455,    70.61820516,  -462.50115691,
         651.57490075])

In [45]:
lr = trained_models['LinearRegression']

In [46]:
lr.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])